In [9]:
import yfinance as yf
import pandas as pd
#import talib as tb
import numpy as np
import ta
from lxml import etree
import matplotlib.pyplot as plt
import seaborn as sns
from spare_parts import get_company_list
import pickle
import warnings
warnings.filterwarnings('ignore')

In [24]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
def balance_sheet(ticker):
    data_1 = pd.read_csv("tickertape.csv")
    x = data_1[data_1['companies_list'] == ticker]['companies']
    x = x.values[0]

    data_1 = urlopen("https://www.tickertape.in/stocks/{}#financials".format(x))
    #print(data_1)
    bsObj = BeautifulSoup(data_1)
    target_area = bsObj.find("div",{"class": "jsx-2510798477"}).find("div", {"class": "jsx-2510798477 financials-table relative mb32"}) #type: ignore
    columns = []
    for i in target_area.find("thead").find_all("th"): #type: ignore
        columns.append(i.text)
    list_1 = []
    final_list = []
    count = 0
    for i in target_area.find("table"): #type: ignore
        if(count == 1):
            temp_list = []
            for j in i.find_all("td"):
                try:
                    x = j.text
                    x = x.replace(",","")
                    if(x == "—"):
                        temp_list.append(x)
                    else:
                        x = float(x)
                        temp_list.append(x)
                except:
                    final_list.append(temp_list)
                    temp_list = []
                    if(x != "—"):
                        list_1.append(x) #type: ignore
        count += 1
    list_1.pop()
    final_list.pop(0)
    #print(final_list)
    print(len(final_list), len(list_1), len(columns))
    data_1 = pd.DataFrame(final_list,index=list_1,columns=columns[1:])
    return data_1

In [25]:
balance_sheet("reliance industries")

15 15 11


,,,,,,,,,,
Total Revenue,384048.00,286288.00,314717.00,401685.00,577698.00,610806.00,505153.00,718061.00,891294.00,891391.00
Raw Materials,292563.00,186254.00,217518.00,276076.00,399167.00,410288.00,301765.00,496369.00,736603.00,736700.00
Power & Fuel Cost,13261.00,10741.00,11251.00,14569.00,17029.00,15098.00,13214.00,17902.00,NaN,NaN
Employee Cost,6262.00,7407.00,8388.00,9523.00,12488.00,14075.00,14817.00,18775.00,NaN,NaN
Selling & Administrative Expenses,12548.00,18808.00,18520.00,29111.00,50221.00,62622.00,56154.00,63557.00,NaN,NaN
Operating & Other expenses,13437.00,9085.00,3511.00,-1778.00,6137.00,10887.00,15981.00,-7065.00,NaN,NaN
EBITDA,45977.00,53993.00,55529.00,74184.00,92656.00,97836.00,103222.00,128523.00,154691.00,154691.00
Depreciation/Amortization,11547.00,11565.00,11646.00,16706.00,20934.00,22203.00,26572.00,29797.00,40319.00,40319.00
PBIT,34430.00,42428.00,43883.00,57478.00,71722.00,75633.00,76650.00,98726.00,114372.00,114372.00
Interest & Other Items,3316.00,3691.00,3849.00,8052.00,16495.00,22027.00,21189.00,14584.00,19571.00,19571.00


In [ ]:
data_raw.columns

In [ ]:
x = [1,2,3,4,5,6,7,8,9,10] # current_count = 8
threshold = 7
current_count = 8
if(current_count <  threshold):
    print("calculation not possible")
else:
    print(x[current_count] - x[current_count-threshold])


In [ ]:
threshold = 7
current_count = 0
actions  = []
logs = []
data_raw_list = data_raw[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].to_numpy()
for i in range(len(data_raw_list)):
    current_price = data_raw_list[i][3]  # closing price
    if(i < threshold):
        continue
    else:
        last_seven_days_price = data_raw_list[i-threshold][3]
        print((current_price-last_seven_days_price)/current_price,"-")
        if(current_price - last_seven_days_price/current_price > 0.1 and current_position_number <=  2):
            current_position_number += 1
            print("bought stock at {}".format(current_price))
            buy_date = data_raw.index[i]
            logs.append(current_price)

        # selling the stock with profit
        if(len(logs) != 0):
          
            max_log_price = max(logs)
            if((current_price - max_log_price) / current_price > 0.2):
                print("sold stock at {} in profit".format(current_price))
                logs.pop(logs.index(max_log_price))
                current_position_number -= 1
                sell_date = data_raw.index[i]
                # appending all the action to main list
                actions.append([sell_date, current_price - max_log_price,"profit"])
        if(len(logs) != 0):
            max_log_price = max(logs)
            if((current_price - max_log_price) / current_price < -0.1):
                print("sold stock at {} in loss".format(current_price))
                logs.pop(logs.index(max_log_price))
                current_position_number -= 1
                sell_date = data_raw.index[i]
                # appending all the action to main list
                actions.append([sell_date, current_price - max_log_price, "loss"])
    

            

        
            
            


           
    
    



In [ ]:
data_raw

In [ ]:
# converting list to dataframe
refined_data =  pd.DataFrame(actions, index = None, columns = ['date', 'profit/loss', 'outcome'])
refined_data.index = refined_data['date']  # type: ignore
refined_data.drop(['date'], axis = 1, inplace = True)

In [ ]:
refined_data['outcome'].value_counts()

In [ ]:
# checking net profit- loss
refined_data['profit/loss'].sum()

<h1> Trend Following Strategies</h1>

In [ ]:
""" 
['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'ma_50','ma_200', 'ma_50_200']
"""
class ma_trading_strategy:
    def __init__(self, ticker):  # can  use ecternal dataset if provided
        # values can be changed here
        self.data_list = self.download_data(ticker)
        self.MAX_PROFIT_LIMIT = 0.2
        self.ticker = ticker
        self.MAX_LOSS_LIMIT =  -0.2
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []

    def download_data(self, ticker, interval = "1d"):
        data_raw = yf.download(ticker ,period = "max" , progress=True, interval=interval)
        # adding ma_50 and ma_200
        data_raw['ma_50'] = ta.trend._ema(data_raw['Close'], 50)  # type: ignore
        data_raw['ma_200'] = ta.trend._ema(data_raw['Close'], 200) # type: ignore
        data_raw['ma_20'] = ta.trend._ema(data_raw['Close'], 20) # type: ignore

        data_raw.dropna(inplace = True)
        data_raw['Date']  =data_raw.index
        data_raw['ma_50_200'] = data_raw['ma_50'] > data_raw['ma_200']
        data_raw['ma_20_200'] = data_raw['ma_20'] > data_raw['ma_200']

        data_raw = data_raw.reindex(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 
                                               'Volume', 'ma_20','ma_50','ma_200','ma_20_200','ma_50_200'])

        #print(data_raw.columns)
        #print(data_raw.index)
        return  data_raw.to_numpy()
    def print_data(self):
        print(self.data_list)
    # main funciton

    def run_test(self):
        for i in range(len(self.data_list)):
            #print(i)
            try:
                # buying section
                current_node = self.data_list[i]
                previous_node = self.data_list[i-1]
                #print(current_node[11], previous_node[11])
                if(self.current_position <= 2 and [previous_node[11],current_node[11]] == [False, True]):
                    self.current_position += 1
                    self.positions.append(current_node[4])
                
                # selling section
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    # selling the stock with profit
                    if((current_node[4] - max_position) / current_node[4]  >= self.MAX_PROFIT_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # apping all the events in logs
                        self.logs.append([current_node[0],current_node[4], net_output, "profit"])
                    # selling the stock with loss
                    if((current_node[4] - max_position) / current_node[4]  < self.MAX_LOSS_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # appending the events in logs
                        self.logs.append([current_node[0],current_node[4], net_output, "loss"])
                
            except:
                pass
    
    # extension functions
    def get_output(self):
        return self.output
    
    def get_logs(self):
        return self.logs


            


In [ ]:
dir(ta)

In [ ]:
dir(ta.volatility)

In [ ]:
help(ta.volatility.average_true_range)

In [ ]:
ta.volatility.UlcerIndex(data_raw['Close'],fillna=False).ulcer_index()

In [ ]:
def download_data(ticker, interval = "1d"):
        data_raw = yf.download(ticker ,period = "max" , progress=True, interval=interval)
        # adding ma_50 and ma_200
        data_raw['ma_50'] = ta.trend._ema(data_raw['Close'], 50)  # type: ignore
        data_raw['ma_200'] = ta.trend._ema(data_raw['Close'], 200) # type: ignore
        data_raw['ma_20'] = ta.trend._ema(data_raw['Close'], 20) # type: ignore
        adx_gen = ta.trend.ADXIndicator(data_raw['High'], data_raw['Low'], data_raw['Close']) 
        data_raw['adx'] = adx_gen.adx()
        data_raw['adx_pos'] = adx_gen.adx_pos()
        data_raw['adx_neg'] = adx_gen.adx_neg()
        data_raw['obv'] = ta.volume.on_balance_volume(data_raw['Close'], data_raw['Volume'],fillna=False)
        # volatility data
        data_raw['ulcer_index'] = ta.volatility.UlcerIndex(data_raw['Close'],fillna=False).ulcer_index()
        alfa = ta.trend.MACD(data_raw['Close'], window_fast=9, window_slow=26) # type: ignore
        macd = alfa.macd()
        macdsignal = alfa.macd_signal()
        data_raw['macd'] = macd
        data_raw['macdsignal'] = macdsignal

        
        data_raw.dropna(inplace = True)
        data_raw['signal']  = data_raw['macd'] > data_raw['macdsignal']
        data_raw['Date']  =data_raw.index
        data_raw['ma_50_200'] = data_raw['ma_50'] > data_raw['ma_200']
        data_raw['ma_20_200'] = data_raw['ma_20'] > data_raw['ma_200']

        data_raw = data_raw.reindex(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'adx','adx_pos','adx_neg','obv','ulcer_index','macd',
                                               'macdsignal','signal',
                                               'Volume', 'ma_20','ma_50','ma_200','ma_20_200','ma_50_200'])
        return data_raw

In [ ]:
download_data("TCS.NS")

In [ ]:
ma_trading_strategy("TCS.NS").run_test()

In [ ]:
# this class sells the stock when another [50-200] crossover happens
class ma_trading_strategy_2(ma_trading_strategy):
    def __init__(self, ticker):
        super().__init__(ticker)
        # addintional attributes
        
        self.experiment = 0
    def run_test(self):
        for i in range(len(self.data_list)):
            # buying stock if condition satisfies
            try:
                current_node = self.data_list[i]
                previous_node = self.data_list[i-1]
                if(self.current_position <= 2 and [previous_node[11],current_node[11]] == [False, True]):
                    self.current_position += 1
                    self.positions.append(current_node[4])
                
                # selling stock if condition satisfies
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    # selling the stock with profit
                    if([previous_node[11], current_node[11]] == [True, False]):
                        self.current_position -= 1
                        self.positions.pop(self.positions.index(max_position))
                        self.output += current_node[4] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], current_node[4] - max_position))
                        self.logs.append([current_node[0],current_node[4], current_node[4] - max_position, "profit"])
                    # selling the stock with loss
                    if((current_node[4] - max_position) / current_node[4]  < self.MAX_LOSS_LIMIT):  # applying - loss limit
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # appending the events in logs
                        self.logs.append([current_node[0],current_node[4], net_output, "loss"])
            except:
                pass
            
                    
                
                



<h1>  Trend following using Momentum strateies </h1>

In [57]:
# Macd based croosover strategy and exiting after machieving some percentage of profit
class macd_strategy:
    def __init__(self, ticker):
        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.05
        self.MAX_LOSS_LIMIT =  -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        self.data_list = None
        self.loss_count  = 0
        self.profit_count = 0
        # applying machine learning capability
    # basic info  function
    def info(self):
        return self.data_list

    # downloading the data for a given ticker
    def download_data(self,ticker, interval = "30m"):
        
        data_raw = yf.download(ticker ,period = "60d" , progress=True, interval=interval)
        # adding ma_50 and ma_200
        data_raw['ma_50'] = ta.trend._ema(data_raw['Close'], 50)  # type: ignore
        data_raw['ma_200'] = ta.trend._ema(data_raw['Close'], 200) # type: ignore
        data_raw['ma_20'] = ta.trend._ema(data_raw['Close'], 20) # type: ignore
        adx_gen = ta.trend.ADXIndicator(data_raw['High'], data_raw['Low'], data_raw['Close'])  #type:   ignore
        data_raw['adx'] = adx_gen.adx()
        data_raw['adx_pos'] = adx_gen.adx_pos()
        data_raw['adx_neg'] = adx_gen.adx_neg()
        data_raw['obv'] = ta.volume.on_balance_volume(data_raw['Close'], data_raw['Volume'],fillna=False) #type: ignore
        # volatility data
        data_raw['ulcer_index'] = ta.volatility.UlcerIndex(data_raw['Close'],fillna=False).ulcer_index() #type: ignore
        alfa = ta.trend.MACD(data_raw['Close'], window_fast=9, window_slow=26) # type: ignore
        macd = alfa.macd()
        macdsignal = alfa.macd_signal()
        data_raw['macd'] = macd
        data_raw['macdsignal'] = macdsignal

        
        data_raw.dropna(inplace = True)
        data_raw['signal']  = data_raw['macd'] > data_raw['macdsignal']
        data_raw['Date']  =data_raw.index
        data_raw['ma_50_200'] = data_raw['ma_50'] > data_raw['ma_200']
        data_raw['ma_20_200'] = data_raw['ma_20'] > data_raw['ma_200']

        data_raw = data_raw.reindex(columns = ['Date', 'Open', 'High', 'Low', 'Close', 
                                               'Adj Close', 'adx','adx_pos',
                                               'adx_neg','obv','ulcer_index','macd',
                                               'macdsignal','signal',
                                               'Volume', 'ma_20','ma_50','ma_200','ma_20_200',
                                               'ma_50_200'])
        return data_raw.to_numpy()
    
    # main funciton
    def run_test(self):

        # machine learning suggessation
       
        self.data_list = self.download_data(self.ticker)
        #print(len(self.data_list))
        for i in range(len(self.data_list)):
      
            try:
                current_node = self.data_list[i]
                previous_node = self.data_list[i-1]
                # buying section
                #print(current_node)
                #[previous_node[13],current_node[13]] == [False, True]
                if(self.current_position <= 2 and [previous_node[13],current_node[13]] == [False, True]
                    ):
                    print(current_node[4])
                    self.current_position += 1
                    self.positions.append(current_node[4])

                    
                # selling section
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    # selling the stock with profit
                    if((current_node[4] - max_position) / current_node[4]  >= self.MAX_PROFIT_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[4], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # apping all the events in logs
                        #self.logs.append([current_node, net_output, "profit"])
                        self.profit_count += 1
                    # selling the stock with loss
                    if((current_node[4] - max_position) / current_node[4]  < self.MAX_LOSS_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[4], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # appending the events in logs
                        #self.logs.append([current_node, net_output, "loss"])
                        self.loss_count += 1
            except:
                pass
        
    
    def get_output(self):
        return self.output
    
            
            





In [48]:
test_1 = macd_strategy("RELIANCE.NS")
test_1.run_test()

[*********************100%***********************]  1 of 1 completed
2345.0
2361.35009765625
2378.5
selling the stock at 2504.0 with profit 125.5
selling the stock at 2504.300048828125 with profit 142.949951171875
selling the stock at 2499.85009765625 with profit 154.85009765625
2479.300048828125
2444.300048828125
2463.89990234375


In [ ]:
test_1.get_output()

In [ ]:
test_1.loss_count

In [ ]:
# creating new strategy
class bollinger_band:
    def __init__(self, ticker):
        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.2
        self.MAX_LOSS_LIMIT = -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        # special - signal 
        self.signal = 0 # this signal changes once stock reaches the upper - bollinger band
    # define function
    def download_data(self):
        data_raw = yf.download(self.ticker+".NS", period="max", interval = "1d")
        # calculating bollinger band
        volatile = ta.volatility.BollingerBands(data_raw['Close']) # type: ignore
        data_raw['bollinger_hband'] = volatile.bollinger_hband()
        data_raw['bollinger_lband'] = volatile.bollinger_lband()
        data_raw['bollinger_mavg'] = volatile.bollinger_mavg()
        data_raw['bollinger_pband'] = volatile.bollinger_pband()
        # calculating obv
        volatile_obv = ta.volume.OnBalanceVolumeIndicator(data_raw['Close'], data_raw['Volume']) #type: ignore
        data_raw['obv'] = volatile_obv.on_balance_volume()
        # calculating rsi
        data_raw['rsi'] = ta.momentum.RSIIndicator(data_raw['Close']).rsi() #type: ignore
        data_raw.dropna(inplace = True)
        data_raw['diff'] = data_raw['obv'].diff(periods=10) # slope
        print(data_raw.columns)

        self.data_list = data_raw.to_numpy()
        return data_raw
    # main function
    def run_test(self):
        self.download_data()
        for i in range(len(self.data_list)):
            # --
            try:
                current_node = self.data_list[i]
                # checking the bollinger signal 
                if(self.signal == 0):
                    if(current_node[6] >=  current_node[3]):
                        self.signal = 1
                # buying section
                if(self.current_position <= 2 and self.signal == 1):
                    # checking [RSI] and [OBV - diff] condition
                    if(current_node[12] > 0 and current_node[11] > 50):
                        self.current_position += 1
                        self.positions.append(current_node[3])
                        self.signal = 0
                # selling section

                # selling with profit condition
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    #(current_node[3] - max_position) / current_node[3]  >= self.MAX_PROFIT_LIMIT
                    if(current_node[3] < current_node[7]):
                        net_output = current_node[3] - max_position
                        print("selling the stock at {} with profit/loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output
                        self.logs.append([current_node[0],current_node[3], net_output, "profit"])
                # selling with loss condititon
            except:
                pass
    
    def get_output(self):
        return self.output


<h1> RSI based strategy </h1>

In [28]:
class rsi_trading():
    def __init__(self, ticker):

        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.2
        self.MAX_LOSS_LIMIT =  -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        self.data_list = self.download_data_1()
    

    def download_data_1(self):
        data_raw = yf.download(self.ticker+".NS" , period = "max", interval = "30min")
        data_raw['Date'] = data_raw.index  #type: ignore
        data_raw['rsi'] = ta.momentum.rsi(data_raw['Close']) #type: ignore
        data_raw.dropna(inplace = True)
        print(data_raw.columns)
        self.data_list = data_raw.to_numpy()
        return data_raw.to_numpy()
    
    # getting data lsit info
    
    def info(self):
        return self.data_list
    
    # buying and selling the stocks
    def run_test(self):
        for i in range(len(self.data_list)):
            try:
                current_node = self.data_list[i]
                #previous_node = self.data_list[i-1]
                # buying section
                if(self.current_position <= 2 and current_node[7] < 30):
                    self.current_position += 1
                    self.positions.append(current_node[4])
                
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    if((current_node[4] - max_position) / current_node[4]  >= self.MAX_PROFIT_LIMIT):

                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output
                    # selling the stock if loss occured
                    if((current_node[4] - max_position) / current_node[4]  < self.MAX_LOSS_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        self.logs.append([current_node[0],current_node[4], net_output, "loss"])
            
            except:
                pass
        
    
    def get_output(self):
        return self.output
        
        

In [ ]:
alfa = rsi_trading("TCS")
alfa.run_test()

In [ ]:
alfa.get_output()

<h1> Playground </h1>

In [ ]:
company_list = get_company_list()
len(company_list)

In [ ]:
company_list

In [ ]:
net_output = {}
for i in company_list[:500]:
    try:

        data = macd_strategy(i[1]+".NS").run_test()
        final_list = []
        for i in test_1.logs:
            data_1 = []
            for i1 in i[0][1:]:
                data_1.append(i1)
            data_1.append(i[1])
            data_1.append(i[2])
            final_list.append(data_1)
        data_1 = pd.DataFrame(final_list, columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'adx','adx_pos','adx_neg','obv','ulcer_index','macd',
                                               'macdsignal','signal',
                                               'Volume', 'ma_20','ma_50','ma_200','ma_20_200','ma_50_200','profit/loss','outcome'])
        data_previous = pd.read_csv("ml_dataset.csv")
        data_previous = pd.concat([data_previous,data_1],ignore_index=True)
        data_previous.to_csv("ml_dataset.csv", index = False)

        
    except:
        pass
   
    

In [58]:
net_output = {}
for i in company_list[100:300]:

        try:

                data = macd_strategy(i[1]+".NS")
                data.run_test()
                net_output[i[1]] = data.get_output()
        except:
                pass
 

[*********************100%***********************]  1 of 1 completed
14080.400390625
14046.349609375
13893.0
[*********************100%***********************]  1 of 1 completed
1284.199951171875
1271.0
1269.550048828125
selling the stock at 1355.0 with profit 70.800048828125
selling the stock at 1355.5 with profit 84.5
selling the stock at 1351.4000244140625 with profit 81.8499755859375
1324.300048828125
1369.949951171875
1408.449951171875
selling the stock at 1484.449951171875 with profit 76.0
selling the stock at 1505.0 with profit 135.050048828125
selling the stock at 1501.3499755859375 with profit 177.0499267578125
1550.0
1551.8499755859375
[*********************100%***********************]  1 of 1 completed
51.95000076293945
53.5
54.79999923706055
selling the stock at 57.70000076293945 with profit 2.9000015258789062
selling the stock at 57.400001525878906 with profit 3.9000015258789062
selling the stock at 56.95000076293945 with profit 5.0
54.95000076293945
55.150001525878906
54.

In [ ]:
net_output

In [ ]:
net_output.keys()

In [59]:
data_1 = pd.DataFrame()
data_1['company'] = list(net_output.keys())
data_1['net_output'] = list(net_output.values())
data_1['net_output'].sum()

110242.90058422089

In [53]:
data_1['net_output'].sum()

102074.50423240662

In [37]:
data_1[data_1['net_output'] < 0].count()

company       9
net_output    9
dtype: int64

In [ ]:
data_1.to_csv("backtesting/bollinger_band_2.csv")

In [ ]:
data_raw = pd.read_csv("backtesting/macd_strategy.csv")


In [ ]:
data_raw['net_output'].sum()

In [ ]:
data_1[data_1['net_output'] < 0].count()

In [ ]:
sns.heatmap(data_1.corr(), annot=True)

In [ ]:
alfa = data_1.corr(method="pearson")


In [ ]:
alfa

In [ ]:
alfa[alfa > .90].fillna("gama", inplace = True)

In [ ]:
alfa.index

In [ ]:
alfa.columns

In [ ]:
alfa.to_numpy()

In [ ]:
# getting companies of custom correlation pair

""" 
pair- selection rate -> .70
"""
data_1 = list(alfa.columns)
data_2 = list(alfa.index)
pairs = []
count = -1
for i in alfa.to_numpy():
    count += 1
    
    for i1 in range(len(list(i))):
        # print(i[i1])

         
         if(i[i1] >= .70 and i[i1] != 1.0):
                #sprint(i[i1])
                try:
                    pairs.append((data_1[i1],data_2[count],i[i1]))
                     
                except:
                     
                    pass
     
pairs

In [ ]:
# creating dataframe out of refined data
pair_trading = pd.DataFrame(pairs, columns=['companies_1', 'companies_2', 'correlation'])
pair_trading.drop_duplicates(inplace = True)

In [ ]:
icicibank_data= yf.download("ICICIBANK.NS", period= "1y", interval = "1d")
hdfcbank_data = yf.download("HDFCBANK.NS", period="1y", interval = "1d")
plt.figure(figsize = (20, 10))
plt.plot(icicibank_data['Close'])
plt.plot(hdfcbank_data['Close'])

In [ ]:
icicibank_data.to_csv("icic.csv")
hdfcbank_data.to_csv("hdfc.csv")

In [ ]:
# plotting the data
new_data = yf.download("SBIN.NS", period ="5y" , interval="1d")
plt.figure(figsize=(20,10))
plt.plot(new_data['Close'])
plt.plot(tb.MA(new_data['Close'], timeperiod=100)) # type: ignore

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(new_data['Close'])
plt.plot(tb.MA(new_data['Close'], timeperiod=50)) # type: ignore

In [ ]:
# applying visulization and anaysis
new_dataframe = pd.DataFrame(net_output, columns = ['company', 'net_output'])
#new_dataframe.to_csv("simple_ma_strategy.csv")

In [ ]:
new_dataframe['net_output'].sum()

In [ ]:
new_dataframe[new_dataframe['net_output'] < 0].count()

<h1> [RSI + OBV + MACD] </h1>

In [ ]:
# generating bollinger bank and obv
data_raw = pd.read_csv("backtesting/tcs_practice_data.csv")
volatile = ta.volatility.BollingerBands(data_raw['Close']) # type: ignore

In [ ]:
volatile_obv = ta.volume.OnBalanceVolumeIndicator(data_raw['Close'], data_raw['Volume']) #type: ignore


In [ ]:
data_raw['obv'] = volatile_obv.on_balance_volume()

In [ ]:
data_raw['bollinger_hband'] = volatile.bollinger_hband()
data_raw['bollinger_lband'] = volatile.bollinger_lband()
data_raw['bollinger_mavg'] = volatile.bollinger_mavg()
data_raw['bollinger_pband'] = volatile.bollinger_pband()

In [ ]:
data_raw.dropna(inplace = True)

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(data_raw['Close'][0:100])
plt.plot(data_raw['bollinger_hband'][0:100])
plt.plot(data_raw['bollinger_lband'][0:100])
plt.legend(['close', 'hband', 'lband'][0:100])
plt.show()
plt.figure(figsize = (20,10))
plt.plot(data_raw['obv'])
plt.show()

In [ ]:
data_raw['diff']  = data_raw['obv'].diff(periods=10)

In [ ]:
list(data_raw['obv'][0:100].diff(periods=10))

In [ ]:
# creating new strategy
class bollinger_band:
    def __init__(self, ticker):
        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.2
        self.MAX_LOSS_LIMIT = -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        # special - signal 
        self.signal = 0 # this signal changes once stock reaches the upper - bollinger band
    # define function
    def download_data(self):
        data_raw = yf.download(self.ticker+".NS", period="max", interval = "1d")
        # calculating bollinger band
        volatile = ta.volatility.BollingerBands(data_raw['Close']) # type: ignore
        data_raw['bollinger_hband'] = volatile.bollinger_hband()
        data_raw['bollinger_lband'] = volatile.bollinger_lband()
        data_raw['bollinger_mavg'] = volatile.bollinger_mavg()
        data_raw['bollinger_pband'] = volatile.bollinger_pband()
        # calculating obv
        volatile_obv = ta.volume.OnBalanceVolumeIndicator(data_raw['Close'], data_raw['Volume']) #type: ignore
        data_raw['obv'] = volatile_obv.on_balance_volume()
        # calculating rsi
        data_raw['rsi'] = ta.momentum.RSIIndicator(data_raw['Close']).rsi() #type: ignore
        data_raw.dropna(inplace = True)
        data_raw['diff'] = data_raw['obv'].diff(periods=10) # slope

        self.data_list = data_raw.to_numpy()
        return data_raw
    # main function
    def run_test(self):
        self.download_data()
        for i in range(len(self.data_list)):
            # --
            try:
                current_node = self.data_list[i]
                # checking the bollinger signal 
                if(self.signal == 0):
                    if(current_node[6] <= current_node[3]):
                        self.signal = 1
                # buying section
                if(self.current_position <= 2 and self.signal == 1):
                    # checking [RSI] and [OBV - diff] condition
                    if(current_node[12] > 0 and current_node[11] > 50):

                        self.current_position += 1
                        self.positions.append(current_node[3])
                        self.signal = 0
                # selling section

                # selling with profit condition
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    if((current_node[3] - max_position) / current_node[3]  >= self.MAX_PROFIT_LIMIT):
                        net_output = current_node[3] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output
                        self.logs.append([current_node[0],current_node[3], net_output, "profit"])
                # selling with loss condititon

                    if((current_node[3] - max_position) / current_node[3]  < self.MAX_LOSS_LIMIT):
                        net_output = current_node[3] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output
                        #self.logs.append([current_node[0],current_node[4], net_output, "loss"])
            except:
                pass
    
    def get_output(self):
        return self.output


            


